# Lecture 10 - Decorators

by Martin Hronec

7th of December

## Decorators
* [nice (very long) tutorial](https://realpython.com/primer-on-python-decorators/) 
* you will encounter decorators sooner rather than later, so you should have an idea what they are and how they work
* also, *metaprogramming* is cool

**Functions (review)**
* to understand *decorators*, quick review of functions in Python
* functions are *first-class objects* (can be passed around and used as arguments, just like any other object)

In [1]:
def x(a,b):
    print(f"param 1: {a}, param 2: {b}")


In [2]:
type(x)

function

In [3]:
# y expects function
def y(z,t):
    """
    z: function
    t: arguments
    """
    z(*t)

In [4]:
y(x,("hello","world"))

param 1: hello, param 2: world


In [5]:
# what if we return the function 
def y_(z,t):
    return(z)

In [6]:
y_(x,("hello","manuel"))

<function __main__.x(a, b)>

* possibility to define *inner functions*
    * the inner functions are not defined until the parent function is called

In [7]:
def parent():
    print("Silence of the parent() function")

    def first_child():
        print("Noise from the first_child() function")

    def second_child():
        print("Signal from the second_child() function")

    second_child()
    first_child()

In [8]:
parent()

Silence of the parent() function
Signal from the second_child() function
Noise from the first_child() function


* they only exist inside the parent() function as local variables
    * try to call `second_child()` 

In [9]:
second_child()

NameError: name 'second_child' is not defined

* we can also return the inner function (reference to it, since we are not running it)

In [10]:
def parent():
    print("Silence of the parent() function")

    def first_child():
        print("Noise from the first_child() function")

    def second_child():
        print("Signal from the second_child() function")

    return(second_child)

In [11]:
papa_func = parent()
papa_func

Silence of the parent() function


<function __main__.parent.<locals>.second_child()>

### Simple decorators

In [12]:
def my_decorator(func):
    def wrapper():
        print("Action-holder BEFORE the function is called.")
        func()
        print("Action-holder ALFTER the function is called.")
    return wrapper

def say_smthng():
    print("Not enough coffee.")

In [13]:
# this is where the decoration happens
beg_for_coffee = my_decorator(say_smthng)

* `say_smthng` points to the `wrapper()` (inner function )
    * we return `wrapper` as a function, when calling my_decorator
* decorators wrap a function, modifying its behavior

In [14]:
beg_for_coffee()

Action-holder BEFORE the function is called.
Not enough coffee.
Action-holder ALFTER the function is called.


In [15]:
from datetime import datetime

def not_after_six(func):
    def wrapper():
        if 5 <= datetime.now().hour < 19:
            func()
        else:
            pass  # Don't drink coffee af night (if you do, do it with bacon only)
    return wrapper

def say_smthng():
    print("Not enough coffee.")

say_smthng = not_after_six(say_smthng)

In [16]:
say_smthng() 

Not enough coffee.


* decoration process above is a little ugly
    * use `@` - *pie syntax*

In [17]:
def my_decorator(func):
    def wrapper():
        print("Action-holder BEFORE the function is called.")
        func()
        print("Action-holder ALFTER the function is called.")
    return wrapper

@my_decorator
def say_smthng():
    print("Not enough coffee.")

* `@my_decorator` is just an easier way of saying `say_whee = my_decorator(say_whee)`
    * this is how we apply a decorator to a function
* decorators are just a regular Python functions 

In [33]:
# another decorator example
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

In [34]:
@do_twice
def say_smthng():
    print("Not enough coffee.")

In [35]:
say_smthng()

Not enough coffee.
Not enough coffee.


**Decorating functions with arguments**

* what if the inner function does not take any argument, but we pass something to it?

In [36]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

In [37]:
greet("World")

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

* use `*args` (Non Keyword Arguments) and `**kwargs` (Keyword Arguments) in the inner wrapper function
    * it will accept an arbitrary number of positional and keyword arguments
    

In [39]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

* `wrapper_do_twice()` inner function now accepts any number of arguments and passes them on to the function it decorates.

In [40]:
greet("World")

Hello World
Hello World


**return value of decorated functions**
* what happens to the return value of decorated function
    * up to the decorator to decide

In [41]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [42]:
hi_tom = return_greeting("Tom")
print(hi_tom)

Creating greeting
Creating greeting
None


* `None` ?
    * reason is that `do_twice_wrapper() doesn't explicitly return a value
* **make sure the wrapper function returns the return value of the decorated function**

In [43]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [28]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [29]:
return_greeting("Tom")

Creating greeting
Creating greeting


'Hi Tom'

### functools.wraps()
* when using *decorators* you technically replace one function `f` with another `f_decor`
    * in the process you however also replace
        * name
        * docstring

In [48]:
def logged(func):
    def logging_wrapper(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return logging_wrapper

In [49]:
@logged
def f(x):
   """not so elaborate docstring"""
   return x + x * x

In [50]:
print(f.__name__)

logging_wrapper


In [51]:
print(f.__doc__)

None


* if using a decorator always meant losing this information about a function, it would be a serious problem
* `functools.wraps` to the rescue
    * takes a function used in a decorator and adds the functionality of copying over the function name, docstring, arguments list, etc. 
* since `wraps` is itself a decorator, the following code works nicely

In [53]:
from functools import wraps
def logged(func):
    @wraps(func)
    def logging_wrapper(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return logging_wrapper

@logged
def f(x):
   """not so elaborate docstring"""
   return x + x * x

print(f.__name__)  # prints 'f'
print(f.__doc__)   # prints 'does some math'

f
not so elaborate docstring


### Example: Timing decorator
* measure the time a function takes to execute

In [54]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      
        run_time = end_time - start_time   
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [55]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0065 secs


In [56]:
waste_some_time(999)

Finished 'waste_some_time' in 4.9705 secs


### Example: Debugging decorator

In [57]:
from functools import wraps

def debug(func):
    """Print the function signature and return value"""
    @wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  
        signature = ", ".join(args_repr + kwargs_repr)           
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")  
        return value
    return wrapper_debug

In [58]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [59]:
make_greeting("Barbora")

Calling make_greeting('Barbora')
'make_greeting' returned 'Howdy Barbora!'


'Howdy Barbora!'

In [60]:
make_greeting(name="Nathatneal", age=116)

Calling make_greeting(name='Nathatneal', age=116)
'make_greeting' returned 'Whoa Nathatneal! 116 already, you are growing up!'


'Whoa Nathatneal! 116 already, you are growing up!'

## Fancy decorators

### Decorating classes
* two main ways how to use decorators on classes:
    * **decorate the methods of a class**
        * [original motivation](https://www.python.org/dev/peps/pep-0318/#motivation) for introducing decorators
        * Python comes with several built-in decorators, the big three are:
            * `@classmethod`
            * `@staticmethod`
            * `@property`
        
    * **decorate the whole class**
        * meaning is similar to the function decorators
        * e.g. [dataclasses module](https://realpython.com/python-data-classes/) in Python 3.7
            * similiarity to metaclasses(advanced)
        
Let’s define a class where we decorate some of its methods using the `@debug` and `@timer` decorators from earlier:

In [21]:
class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [22]:
tw = TimeWaster(1000)

Calling __init__(<__main__.TimeWaster object at 0x7f60fc207d60>, 1000)
'__init__' returned None


In [23]:
tw.waste_time(999)

Finished 'waste_time' in 0.1849 secs


### Nesting decorators
* stack decorators on top of each other:
    *  decorators being executed in the order they are listed.

In [25]:
@debug
@do_twice
def greet(name):
    print(f"Hello {name}")

NameError: name 'do_twice' is not defined

In [26]:
greet("Tomas")

NameError: name 'greet' is not defined

In [27]:
@do_twice
@debug
def greet(name):
    print(f"Hello {name}")

NameError: name 'do_twice' is not defined

In [28]:
greet("Tomas")

NameError: name 'greet' is not defined

### Decorators with arguments
* until now, decorators were argumentless

In [29]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [30]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

In [31]:
greet('Lenka')

Hello Lenka
Hello Lenka
Hello Lenka
Hello Lenka


### Example: Slowing down code
* we will make decorator `@slow_down` callable both with and without arguments

In [33]:
import functools
import time

def slow_down(_func=None, *, rate=1):
    """Sleep given amount of seconds before calling the function"""
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)

In [34]:
@slow_down(rate=1)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [247]:
countdown(3)

3
2
1
Liftoff!
